In [ ]:
!pip install aiolimiter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --upgrade GoogleNews

In [36]:
from bs4 import BeautifulSoup
import urllib.request,sys,time
import requests
import pandas as pd
import dateutil.parser as dparser
from googlesearch import search
from datetime import date, timedelta, datetime
import numpy as np
import asyncio
import aiohttp
import attr
import time
from aiolimiter import AsyncLimiter
# import re
# from serpapi import GoogleSearch
# from GoogleNews import GoogleNews
# import selenium.webdriver as webdriver

In [1]:
## use simple request instead of googlesearch
# https://github.com/Fryingpannn/WallStreetBets_BigDataAnalysis

In [37]:
START_DATE = date(2019,2,12) 
END_DATE = date(2020,2,1)

In [38]:
### @param url: takes url of news page of moneycontrol website
### @ returns list of articles on the page
def get_urls_from_page(url):
  try:
     page=requests.get(url) 
  except Exception as e:    
    error_type, error_obj, error_info = sys.exc_info()  
    print (error_type,'ERROR FOR LINK:',url, 'Line:', error_info.tb_lineno) 

  if page.status_code != 200:
    return None

  soup = BeautifulSoup(page.text, "html.parser")
  section = soup.find('section',attrs={'class':'mid-contener contener clearfix'})
  ul = section.find('ul')
  li_tags=ul.find_all('li',attrs={'class':'clearfix'})

  article_urls = []
  for li in li_tags:
    a_tag = li.find('a')
    link = a_tag.get('href')
    article_urls.append(link)

  return article_urls


In [39]:
## getting content of each article

def parse_urls(article_urls):
  data = []

  for url in article_urls:
    try:
        article = requests.get(url)
        time.sleep(2)
        row = parse_article(article)
        data.append(row) 

    except Exception as e:    
      continue

  return pd.DataFrame(data)

In [40]:
### @param article : result of requests.get()
### @ returns title,date,description,content of the moneycontrol aritcles
def parse_article(article):

  article = BeautifulSoup(article.text,"html.parser")

  article_title = str(article.find('h1',attrs={'class':'article_title artTitle'}).string)
  article_desc = str(article.find('h2',attrs={'class':'article_desc'}).string)

  article_datetime =  article.find('div',attrs={'class':'article_schedule'})
  article_datetime = article_datetime.span.string + str(article_datetime.contents[2])
  article_datetime = dparser.parse(article_datetime)


  article_p_tags = article.find('div',attrs={'class':'content_wrapper arti-flow'}).find_all('p')    

  for p_tag in article_p_tags:
    while p_tag.img is not None:
      p_tag.img.decompose()

    while p_tag.a is not None:
      text = p_tag.a.string
      p_tag.a.replace_with(text)

  article_content = ''.join([str(p_tag.string) for p_tag in article_p_tags])

  return {'title':article_title,
          'datetime':article_datetime,
          'description':article_desc,

          'content':article_content}



In [41]:
def url_finder(start_date,end_date):
  start_date = str(start_date)
  end_date = str(end_date)
  query = f'site:moneycontrol.com after:{start_date} before:{end_date}'
  links = search(query)

  urls = []
  while True:
    time.sleep(5)
    try:
        link = next(links)
        if link.startswith('https://www.moneycontrol.com/news/'):
          urls.append(link)
          # print("link added")
    except StopIteration:
        break
  
  return urls

In [42]:
def get_moneycontrol_links():

  start_date = START_DATE
  end_date  = start_date + timedelta(days=7)
  results = []

  while end_date < END_DATE:
    print(f'Getting Links from {str(start_date)} to {str(end_date)}')
    
    urls = url_finder(start_date,end_date)
    print(f'got {len(urls)} links now sleeping')
    time.sleep(50)
    
    results.append(urls)

    start_date += timedelta(days=7)
    end_date += timedelta(days=7)

  # result = asyncio.gather(*results)
  return results

links = get_moneycontrol_links()

Getting Links from 2019-02-12 to 2019-02-19


HTTPError: ignored

## Using semaphore and rate limiter

In [ ]:
@attr.s
class Fetch :
   limit = attr.ib() #batch
   rate = attr.ib(default = 1, converter= int) #speed
  
   async def make_request (self,query) :
      news_links = []

      async with self.limit :
          async with aiohttp.ClientSession() as session :

            links = search(query)
            while True:
              try:
                  link = next(links)
                  if link.startswith('https://www.moneycontrol.com/news/'):
                    news_links.append(link)
              except StopIteration:
                  break
                
            await asyncio.sleep (self.rate)

      return news_links


In [ ]:
async def fetch_urls() :
    limit = asyncio.Semaphore(2)
    f = Fetch (rate = 4,
               limit = limit )
    
    # getting all for api requests
    queries = []
    start_date = START_DATE
    end_date  = start_date + timedelta(days=7)

    while end_date < END_DATE:
      queries.append(f'site:moneycontrol.com after:{start_date} before:{end_date}')
      start_date += timedelta(days=7)
      end_date += timedelta(days=7)
    
    # getting results from all queries
    results = []
    for query in queries :
        await results.append(f.make_request (query = query) )
        # print("Query done :",query)

    result = await asyncio.gather(*results)
    return result

In [ ]:
# try:
#      page=requests.get(url) 
# except Exception as e:    
#     error_type, error_obj, error_info = sys.exc_info()  
#     print ('ERROR FOR LINK:',url)                         
#     print (error_type, 'Line:', error_info.tb_lineno)


# page.status_code

# soup = BeautifulSoup(page.text, "html.parser")
# section = soup.find('section',attrs={'class':'mid-contener contener clearfix'})
# ul = section.find('ul')
# li_tags=ul.find_all('li',attrs={'class':'clearfix'})

# article_urls = []
# for li in li_tags:
#   a_tag = li.find('h2').find('a')
#   link = a_tag.get('href')
#   article_urls.append(link)

# df_final = pd.DataFrame()
# for page in range(1,31):  
#   url = f'https://www.moneycontrol.com/news/business/page-{page}'
#   article_urls = get_urls_from_page(url) 
#   data = parse_urls(article_urls)
#   df_final.append(data,ignore_index=True)

<coroutine object fetch_urls at 0x7f066c573830>